In [112]:
%load_ext autoreload
%autoreload 1

import pandas as pd
import numpy as np

%aimport SweetSpotPrediction
import SweetSpotPrediction as ssp

# 01 Data Preperation
Here we can convert the raw data into our features and labels.

*Paths*
* Classification: Simple datatable for each (X,Y) Pos
<table>
    <thead><tr><td>X pos</td><td>Y pos</td><td>value_map_1</td><td>value_map_2</td><td>value_map_3</td><td>A_well_was_drilled_at_this_pos</td></tr></thead>
    <tr><td>0</td><td>0</td><td>5.2</td><td>1.2</td><td>1.2</td><td>False</td></tr>
    <tr><td>0</td><td>1</td><td>6.3</td><td>6.3</td><td>6.3</td><td>True</td></tr>
    </table>
* Regression: Simple datatable for each (X,Y) Pos. Distance factor (0.0-1.0) should probably be a quadratic factor?
<table>
    <thead><tr><td>X pos</td><td>Y pos</td><td>value_map_1</td><td>value_map_2</td><td>value_map_3</td><td>Distance_factor_from_actual_well</td></tr></thead>
    <tr><td>0</td><td>0</td><td>5.2</td><td>1.2</td><td>1.2</td><td>0.62</td></tr>
    <tr><td>0</td><td>1</td><td>6.3</td><td>6.3</td><td>6.3</td><td>1.0</td></tr>
    </table>
* Deep Learning (**Out of scope**): Combine the 2D maps into a single feature space and as a lable a 2D mask with actual production well location (perfect for convolutional neural networks)

# Read Maps, including Well Probability

In [113]:
# File paths with map names
files_names = {'Data\\thickness\\thickness': 'Thickness'
         , 'Data\\avgntg\\avgntgasc': 'AvgNTG'
         , 'Data\\avgperm\\avgpermasc': 'AvgPerm'
         , 'Data\\avgporo\\avgporoasc': 'AvgPoro'
         , 'Data\\netperm\\netpermasc': 'NetPerm'
         , 'Data\\distwellweight.txt': 'WellProbability'
              }

# Read map files
dfs = []
spacing = 50 # map grid cell dimension
for key in files_names:
    df = pd.read_csv(key, delimiter=' ', comment='#', names=['XPos', 'YPos', files_names[key], 'c1', 'c2'])
    df.drop(columns=['c1', 'c2'], inplace=True)
    df_reg = ssp.make_regular(df, files_names[key], spacing)
    df_reg.set_index(['XPos','YPos'], inplace=True)
    dfs = dfs + [df_reg]

# Merge dataframes
df_cls = pd.concat(dfs, axis='columns', join='inner').reset_index()

# Add Well locations (Classification)

In [114]:
# This is a fudge, for now

df_cls['WellPresent'] = df_cls.WellProbability > 0.75

In [115]:
df_cls.head()

,XPos,YPos,Thickness,AvgNTG,AvgPerm,AvgPoro,NetPerm,WellProbability,WellPresent
0,452000.0,6780600.0,-19.843674,NaN,NaN,NaN,NaN,NaN,False
1,452000.0,6780650.0,-17.164212,NaN,NaN,NaN,NaN,NaN,False
2,452000.0,6780700.0,-14.827628,NaN,NaN,NaN,NaN,NaN,False
3,452000.0,6780750.0,-14.540321,NaN,NaN,NaN,NaN,NaN,False
4,452000.0,6780800.0,-15.951475,NaN,NaN,NaN,NaN,NaN,False


# Data cleaning

# Feature Engineering

# Save the data

In [4]:
df_cls.to_csv('ClassificationData.csv')
df_reg.to_csv('RegressionData.csv')

# Copy paste features and Labels

In [5]:
features = [
    'XPos', # propbably dont want these as features
    'YPos', # propbably dont want these as features
    'Random1',
    'Random2'

]

label_clf = 'WellPresent' 
label_reg = 'WellProbability'

# TODO
* Discuss Train/Validation/Test split (arbitrary random or specific say reserving a few wells for each set)??

# <a href="02 Base Model.ipynb">Next</a> Notebook